In [ ]:
import warnings
warnings.simplefilter("ignore", FutureWarning)


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
from pathlib import Path
import time

# LightGBM
from lightgbm import LGBMClassifier

# sklearn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import (
    train_test_split, KFold, cross_validate, cross_val_score,
    RandomizedSearchCV, StratifiedKFold
)

from sklearn.metrics import (
    classification_report, confusion_matrix,
    accuracy_score, roc_auc_score
)

from sklearn.base import BaseEstimator, TransformerMixin, clone
from scipy.stats import randint, uniform, loguniform

# Importações locais
from setup_notebook import setup_path
setup_path()

from src.model_utils import *
from src.preprocess_utils_diab import *

print("\n#Processo iniciado em:", time.strftime("%H:%M:%S"))
start_inicial = time.time()

In [ ]:
BASE = Path.cwd().parent

PP2 = joblib.load(BASE/'src'/'preprocess_diabetes_v1.2.joblib')['preprocessador']

DATA_DIR = BASE/"data"/"raw"
X_train = pd.read_csv(DATA_DIR/"X_train_raw.csv")
X_test  = pd.read_csv(DATA_DIR/"X_test_raw.csv")
y_train = pd.read_csv(DATA_DIR/"y_train_raw.csv").values.ravel()
y_test  = pd.read_csv(DATA_DIR/"y_test_raw.csv").values.ravel()
mtd_scoring='roc_auc'

In [ ]:
params_search_1 = {
    "colsample_bytree": 0.2558816829190137,
    "learning_rate": 0.029540909142921727,
    "max_depth": 9,
    "min_child_samples": 19,
    "n_estimators": 1981,
    "num_leaves": 108,
    "reg_alpha": 4.433401936490238,
    "reg_lambda": 3,
    "subsample": 0.7498450458505885
}

model_lgbm_s1 = LGBMClassifier(
    objective="binary",
    random_state=42,
    n_jobs=-1,
    verbose=-1,
    force_row_wise=True,
    **params_search_1
)

pipe_s1 = pipe_models(model_lgbm_s1, PP2)


params_search_2 = {
    "colsample_bytree": 0.21763559048609216,
    "learning_rate": 0.025617518329851284,
    "max_depth": 8,
    "min_child_samples": 13,
    "n_estimators": 1986,
    "num_leaves": 76,
    "reg_alpha": 4.075002570150084,
    "reg_lambda": 2.726896430402328,
    "subsample": 0.7933714832397691
}
model_lgbm_s2 = LGBMClassifier(
    objective="binary",
    random_state=42,
    n_jobs=-1,
    verbose=-1,
    force_row_wise=True,
    **params_search_2
)

pipe_s2 = pipe_models(model_lgbm_s2, PP2)

In [1]:
# fiting 
pipe_s1.fit(X_train, y_train)
pipe_s2.fit(X_train, y_train)

NameError: name 'pipe_s1' is not defined

In [ ]:
#Salvar Hiperparametros joblib

joblib.dump(search_1.best_estimator_, 'modelo_XGB_final_randsearch.'+mtd_scoring+'_v1.2.joblib')
joblib.dump(search_2.best_estimator_, 'modelo_XGB_final_refine.'+mtd_scoring+'_v1.2.joblib')